In [1]:
from tradingcore.execution_manager import ExecutionManager
from tradingcore.campaign import Campaign
from tradingcore.account import Account
from tradingcore.moneymanagement import PlainMM

from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.datasource_sql import DataSourceSQL
from exobuilder.data.assetindex_mongo import AssetIndexMongo

import pprint

In [2]:
try:
    from .settings import *
except SystemError:
    from scripts.settings import *

try:
    from .settings_local import *
except SystemError:
    try:
        from scripts.settings_local import *
    except ImportError:
        pass
    pass

In [3]:
assetindex = AssetIndexMongo(MONGO_CONNSTR, MONGO_EXO_DB)
#datasource = DataSourceSQL(SQL_HOST, SQL_USER, SQL_PASS, assetindex, futures_limit=10, options_limit=10)
datasource = DataSourceMongo(MONGO_CONNSTR, MONGO_EXO_DB, assetindex, futures_limit=10, options_limit=10)

In [4]:
exmgr = ExecutionManager(MONGO_CONNSTR, datasource, dbname=MONGO_EXO_DB)

In [15]:
campaigns = exmgr.campaign_load_all()
accounts = exmgr.account_load_all()
pp = pprint.PrettyPrinter(indent=4)

## List of available campaigns

In [16]:
[print(x) for x in campaigns.keys()];

test_campaign


## List of available accounts

In [17]:
[print(x) for x in accounts.keys()];

new_name
new_account1
test_account


# Add new account

In [8]:
acc_name = 'new_account1'
acc_campaign = 'test_campaign'
acc_client = 'test_client'
acc_size_factor = 1.0

In [9]:
acc_dict = {
            'name': acc_name,
            'client_name': acc_client,
            'info': {
                'size_factor': acc_size_factor,
            }
        }

_campaign = campaigns[acc_campaign]
# Account MM class
mmclass = PlainMM(acc_dict['info'])

new_acc = Account(acc_dict, _campaign, mmclass)

pp.pprint(new_acc.as_dict())

{   'campaign_name': 'test_campaign',
    'client_name': 'test_client',
    'info': {'size_factor': 1.0},
    'mmclass_name': 'plain',
    'name': 'new_account1'}


### Save new account to DB

In [10]:
exmgr.account_save(new_acc)

# Single account edit

In [18]:
acc_name = 'test_account'

In [19]:
acc = exmgr.account_load(acc_name)
pp.pprint(acc.as_dict())

{   'campaign_name': 'test_campaign',
    'client_name': 'test_client8',
    'info': {'size_factor': 1.0},
    'mmclass_name': 'plain',
    'name': 'test_account'}


In [20]:
acc.name = 'new_name'
acc.client_name = 'new_client_name'
acc.info = {'size_factor': 1.0}
acc.mmclass = PlainMM(acc.info)

### Save changes

In [21]:
exmgr.account_save(acc)

### Account information

In [22]:
pp.pprint(acc.as_dict())

{   'campaign_name': 'test_campaign',
    'client_name': 'new_client_name',
    'info': {'size_factor': 1.0},
    'mmclass_name': 'plain',
    'name': 'new_name'}
